---

#### $Purpose$ $of$ $the$ $Notebook$ 

---

The goal of this notebook is to generate predictions on several datasets:  

- **QDMR**  
- **HotpotQA**  
- **StrategyQA**  


#### $Predictions$  
- Predictions were generated for **5 examples**.  
- Shot examples: **3**.  

---

#### $Load$ $Libraries$

---

In [ ]:
import json
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, AutoModel
from huggingface_hub import notebook_login
import textwrap
import sys
utils_path = os.path.abspath("../utils")
if utils_path not in sys.path:
    sys.path.append(utils_path)
import re
from run_experiment import run_experiment
from manage_folders import save_results_to_json, dataset_folders
from retriever import DynamicRetriever
from accelerate import infer_auto_device_map, dispatch_model
import random

---

#### $Load$ $Model$

---

##### $Model$ $Access$

In order to access to the model we need to:
1. Visit the webpage of the model: https://huggingface.co/google/gemma-2b-it
2. Log in to our HF account
3. Click on `Terms` 
4. Write yout contact information (email and name)
5. Accept the terms
6. Run `from huggingface_hub import notebook_login` and `notebook_login()` 
7. Create a new token with `read` rights
8. Copy the token and paste it in the notebook and press `Enter`


*It might take some minutes*...


In [ ]:
notebook_login()

In [ ]:
# Initialize the model name 
model_name = "google/gemma-2b-it"

##### $Bnb$ $Configuration$


The `bnb_config` creates a configuration to shrink the language model. 

* *Compresses the Model:* It tells the system to load the model in a "compressed" 4-bit format instead of its full 16-bit size.
* *Saves Memory:* This makes the model about 4 times smaller, allowing it to run on computers with less memory (RAM and VRAM).
* *Maintains Performance:* It uses clever tricks (like doing the actual math in 16-bit) to ensure the shrunken model is still fast and accurate.

Essentially, it's a set of instructions to make a huge model fit on the computer with minimal loss in quality.

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

##### $Tokenizer$ $Set-up$


This code sets up a `tokenizer` for the language model.
It loads a pre-trained tokenizer, makes sure there's a padding token.

In [ ]:
# Load the tokenizer. The library will handle the chat template automatically.
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    print("pad_token not set. Setting it to eos_token for this model.")
    tokenizer.pad_token = tokenizer.eos_token


print(f"Tokenizer for {model_name} loaded successfully.")


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

#### $System$ $Prompt$

To ensure the model performs the intended task—such as question decomposition—we provide clear and detailed instructions. This involves defining the model’s role, outlining the steps it should follow, and specifying the desired structure of the output. By being as explicit as possible, we guide the model toward producing consistent and accurate results.

In [ ]:
system_prompt = (
    "You are an expert at breaking down complex questions into a numbered sequence of simple, factual sub-questions. "
    "After listing all sub-questions DO NOT provide anything else.\n\n"
    
    "Follow this exact format:\n"
    "subq1: <sub-question>\n"
    "subq2: <sub-question>\n"
    "... continue numbering sequentially as needed ...\n"
    
    "Do NOT repeat the original question. "
    "Do NOT include commentary, explanations, assumptions, or any extra text. "
    "Strictly adhere to the numbering and format above."
)


#### $Parameters$ $for$ $the$ $Model$ $Configuration:$

* **`max_new_tokens`**
  Defines the maximum number of tokens the model can generate in a single response.

  * Example: `max_new_tokens=512` means the model will not produce more than 512 tokens beyond the prompt, preventing it from generating endlessly.

* **`do_sample`**
  Controls whether the model generates text deterministically or stochastically.

  * **`do_sample=False`** → Deterministic: the model always picks the most likely next token, producing the same output for the same input. Useful for comparison and evaluation.
  * **`do_sample=True`** → Stochastic: the model samples from the probability distribution, allowing for more variety and creativity. Choices are influenced by parameters such as *temperature* and *top_p*.

  🔹 For **Gemma-2b-it**, we use:

  * `do_sample=True`
  * `temperature=0.7`
  * `top_p=0.9`

<!-- * **`repetition_penalty`**
  Adjusts the likelihood of repeating tokens to encourage more diverse output.

  * Default: `1.0` (no penalty)
  * Typical range: `1.1 – 2.0`

    * ~1.2 → light penalty
    * ≥1.5 → strong penalty -->



In [ ]:
# Define the model specific configuration
model_config = {
    "model_id": model_name,
    "uses_system_prompt": False,  # Gemma doesnt support a system prompt
    "system_prompt": system_prompt,	
        "max_new_tokens": 256,
        "do_sample": True,
        "temperature": 0.7,
		"top_p": 0.9,
        # "repetition_penalty": 1.2
    }


In [ ]:
# Define the name of the model for our file names
model_file_name = "Gemma-2b-it_results.json"

---

#### $QDMR$ $Dataset$ $Predictions$

---

We use a helper function `dataset_folders(base_results_folder, dataset_path, few_shot_examples_path)` that streamlines the setup process for running experiments on a new dataset. It handles three key tasks:

1.  **Creates Output Directories:** It takes a base folder path and automatically creates the `zero_shot` and `few_shot` subdirectories where the model's predictions will be saved.
2.  **Loads Evaluation Data:** It reads the main dataset file (e.g., `hotpot_dataset.json`) containing the questions to be evaluated.
3.  **Loads Few-Shot Examples:** It reads the corresponding file containing the high-quality examples for few-shot prompting.

The function returns a single, convenient dictionary containing all these assets (the loaded data and the output folder paths), which can then be easily used by the main experiment functions.

In [ ]:
qdmr_base_folder = "../QDMR/llm_predictions/static"
qdmr_dataset_file = "../QDMR/QDMR_examples/qdmr_evaluation.json" # Define the path to the QDMR dataset
qdmr_fewshot_file = "../QDMR/QDMR_examples/qdmr_few_shot.json"

# Call the function to get everything we need in one go
data_assets = dataset_folders(qdmr_base_folder, qdmr_dataset_file, qdmr_fewshot_file, few_shot_type="static", tuning_subset_size=5)

# Unpack the dictionary into these variables for easy access
qdmr_data_full = data_assets["data_full"]
qdmr_data = data_assets["data_subset"] # This is the subset of 5 examples for tuning
shot_examples = data_assets["shot_examples"]
zero_shot_folder = data_assets["zero_shot_folder"]
few_shot_folder = data_assets["few_shot_folder"]
    

##### $Zero$ $Shot$ $Experiment$

In [ ]:
# Run the zero-shot experiment
print("\nStarting Zero-Shot Experiment")

zero_shot_results = run_experiment(
    model=model,
    tokenizer=tokenizer,
    data=qdmr_data,
    shot_examples=shot_examples,
    model_config=model_config,  
    num_shots=0,
    batch_size=5
)
save_results_to_json(zero_shot_results, zero_shot_folder, model_file_name)

##### $Few$ $Shot$ $Experiment$ $-$ $Static$ $Shots$

In [ ]:
# Run the few-shot experiment with 3 shots
print("\nStarting 3-Shot Experiment")
# Run a 3-shot experiment
three_shot_results = run_experiment(
    model=model,
    tokenizer=tokenizer,
    data=qdmr_data,
    shot_examples=shot_examples,
    model_config=model_config, # Use the same config
    num_shots=3,
    few_shot_type="static",  # "static" | "random" | "dynamic"
    retriever=None,          # Required if few_shot_type="dynamic"
    seed=42,
    batch_size=5
)

# Save the results with a different filename
save_results_to_json(three_shot_results, few_shot_folder, f"3shot_{model_file_name}")

##### $Few$ $Shot$ $Experiment$ $-$ $Random$ $Shots$

In [ ]:
qdmr_base_folder = "../QDMR/llm_predictions/random"
qdmr_dataset_file = "../QDMR/QDMR_examples/qdmr_evaluation.json" # Define the path to the QDMR dataset
qdmr_fewshot_file = "../QDMR/QDMR_examples/qdmr_few_shot.json"

# Call the function to get everything we need in one go
data_assets = dataset_folders(qdmr_base_folder, qdmr_dataset_file, qdmr_fewshot_file, few_shot_type="static", tuning_subset_size=5)

# Unpack the dictionary into these variables for easy access
qdmr_data_full = data_assets["data_full"]
qdmr_data = data_assets["data_subset"] # This is the subset of 5 examples for tuning
shot_examples = data_assets["shot_examples"]
zero_shot_folder = data_assets["zero_shot_folder"]
few_shot_folder = data_assets["few_shot_folder"]
    

In [ ]:
# Run the few-shot experiment with 3 shots
print("\nStarting Random-3-Shot Experiment")
# Run a 3-shot experiment
three_shot_results = run_experiment(
    model=model,
    tokenizer=tokenizer,
    data=qdmr_data,
    shot_examples=shot_examples,
    model_config=model_config, # Use the same config
    num_shots=3,
    few_shot_type="random",  # "static" | "random" | "dynamic"
    retriever=None,          # Required if few_shot_type="dynamic"
    seed=42,
	batch_size=5
)

# Save the results with a different filename
save_results_to_json(three_shot_results, few_shot_folder, f"3shot_{model_file_name}")

##### $Few$ $Shot$ $Experiment$ $-$ $Dynamic$ $Shots$

In [ ]:
qdmr_base_folder = "../QDMR/llm_predictions/dynamic"
qdmr_dataset_file = "../QDMR/QDMR_examples/qdmr_evaluation.json" # Define the path to the QDMR dataset
qdmr_fewshot_file = "../QDMR/QDMR_examples/qdmr_few_shot.json"

# Call the function to get everything we need in one go
data_assets = dataset_folders(qdmr_base_folder, qdmr_dataset_file, qdmr_fewshot_file, few_shot_type="static", tuning_subset_size=5)

# Unpack the dictionary into these variables for easy access
qdmr_data_full = data_assets["data_full"]
qdmr_data = data_assets["data_subset"] # This is the subset of 5 examples for tuning
shot_examples = data_assets["shot_examples"]
zero_shot_folder = data_assets["zero_shot_folder"]
few_shot_folder = data_assets["few_shot_folder"]
    

In [ ]:
# Run the few-shot experiment with 3 shots
print("\nStarting Dynamic-3-Shot Experiment")

retriever_instance = DynamicRetriever(shot_examples)

# Run a 3-shot experiment
three_shot_results = run_experiment(
    model=model,
    tokenizer=tokenizer,
    data=qdmr_data,
    shot_examples=shot_examples,
    model_config=model_config, # Use the same config
    num_shots=3,
    few_shot_type="dynamic",  # "static" | "random" | "dynamic"
    retriever=retriever_instance,          # Required if few_shot_type="dynamic"
    seed=42,
    batch_size=5
)

# Save the results with a different filename
save_results_to_json(three_shot_results, few_shot_folder, f"3shot_{model_file_name}")

---

#### $HotpotQA$ $Dataset$ $Predictions$

---

In [ ]:
# Define the paths for the dataset we want to use
hotpot_base_folder = '../HotpotQA/llm_predictions/static/'
hotpot_dataset_file = '../HotpotQA/HotpotQA_examples/hotpot_evaluation.json'
hotpot_fewshot_file = '../HotpotQA/HotpotQA_examples/hotpot_few_shot.json'

# Call the function to get everything we need in one go
data_assets = dataset_folders(hotpot_base_folder, hotpot_dataset_file, hotpot_fewshot_file, few_shot_type="static", tuning_subset_size=5)

# Unpack the dictionary into these variables for easy access
hotpot_data_full = data_assets["data_full"]
hotpot_data = data_assets["data_subset"] # This is the subset of 5 examples for tuning
shot_examples = data_assets["shot_examples"]
zero_shot_folder = data_assets["zero_shot_folder"]
few_shot_folder = data_assets["few_shot_folder"]
    
    

##### $Zero$ $Shot$ $Experiment$

In [ ]:
# Run the zero-shot experiment
print("\nStarting Zero-Shot Experiment")

zero_shot_results = run_experiment(
    model=model,
    tokenizer=tokenizer,
    data=hotpot_data,
    shot_examples=shot_examples,
    model_config=model_config,  
    num_shots=0,
    batch_size=5
)

save_results_to_json(zero_shot_results, zero_shot_folder, model_file_name)

##### $Few$ $Shot$ $Experiment$ $-$ $Static$ $Shots$

In [ ]:
# Run the few-shot experiment with 3 shots
print("\nStarting 3-Shot Experiment")
# Run a 3-shot experiment
three_shot_results = run_experiment(
    model=model,
    tokenizer=tokenizer,
    data=hotpot_data,
    shot_examples=shot_examples,
    model_config=model_config, # Use the same config
    num_shots=3,
    few_shot_type="static",  # "static" | "random" | "dynamic"
    retriever=None,          # Required if few_shot_type="dynamic"
    seed=42,
    batch_size=5
)

# Save the results with a different filename
save_results_to_json(three_shot_results, few_shot_folder, f"3shot_{model_file_name}")

##### $Few$ $Shot$ $Experiment$ $-$ $Random$ $Shots$

In [ ]:
# Define the paths for the dataset we want to use
hotpot_base_folder = '../HotpotQA/llm_predictions/random/'
hotpot_dataset_file = '../HotpotQA/HotpotQA_examples/hotpot_evaluation.json'
hotpot_fewshot_file = '../HotpotQA/HotpotQA_examples/hotpot_few_shot.json'

# Call the function to get everything we need in one go
data_assets = dataset_folders(hotpot_base_folder, hotpot_dataset_file, hotpot_fewshot_file, few_shot_type="static", tuning_subset_size=5)

# Unpack the dictionary into these variables for easy access
hotpot_data_full = data_assets["data_full"]
hotpot_data = data_assets["data_subset"] # This is the subset of 5 examples for tuning
shot_examples = data_assets["shot_examples"]
zero_shot_folder = data_assets["zero_shot_folder"]
few_shot_folder = data_assets["few_shot_folder"]
    
    

In [ ]:
# Run the few-shot experiment with 3 shots
print("\nStarting Random-3-Shot Experiment")
# Run a 3-shot experiment
three_shot_results = run_experiment(
    model=model,
    tokenizer=tokenizer,
    data=hotpot_data,
    shot_examples=shot_examples,
    model_config=model_config, # Use the same config
    num_shots=3,
    few_shot_type="random",  # "static" | "random" | "dynamic"
    retriever=None,          # Required if few_shot_type="dynamic"
    seed=42,
    batch_size=5
)

# Save the results with a different filename
save_results_to_json(three_shot_results, few_shot_folder, f"3shot_{model_file_name}")

##### $Few$ $Shot$ $Experiment$ $-$ $Dynamic$ $Shots$

In [ ]:
# Define the paths for the dataset we want to use
hotpot_base_folder = '../HotpotQA/llm_predictions/dynamic/'
hotpot_dataset_file = '../HotpotQA/HotpotQA_examples/hotpot_evaluation.json'
hotpot_fewshot_file = '../HotpotQA/HotpotQA_examples/hotpot_few_shot.json'

# Call the function to get everything we need in one go
data_assets = dataset_folders(hotpot_base_folder, hotpot_dataset_file, hotpot_fewshot_file, few_shot_type="static", tuning_subset_size=5)

# Unpack the dictionary into these variables for easy access
hotpot_data_full = data_assets["data_full"]
hotpot_data = data_assets["data_subset"] # This is the subset of 5 examples for tuning
shot_examples = data_assets["shot_examples"]
zero_shot_folder = data_assets["zero_shot_folder"]
few_shot_folder = data_assets["few_shot_folder"]
    
    

In [ ]:
# Run the few-shot experiment with 3 shots
print("\nStarting Dynamic-3-Shot Experiment")

retriever_instance = DynamicRetriever(shot_examples)

# Run a 3-shot experiment
three_shot_results = run_experiment(
    model=model,
    tokenizer=tokenizer,
    data=hotpot_data,
    shot_examples=shot_examples,
    model_config=model_config, # Use the same config
    num_shots=3,
    few_shot_type="dynamic",  # "static" | "random" | "dynamic"
    retriever=retriever_instance,          # Required if few_shot_type="dynamic"
    seed=42,
	batch_size=5
)

# Save the results with a different filename
save_results_to_json(three_shot_results, few_shot_folder, f"3shot_{model_file_name}")

---

#### $StrategyQA$ $Dataset$ $Predictions$

---

In [ ]:
# Create the folder paths for results and the folders if they do not exist
strategyqa_base_folder = '../StrategyQA/llm_predictions/static'
strategyqa_dataset_file = "../StrategyQA/StrategyQA_examples/strategyqa_evaluation.json" # Define the path to the strategyqa dataset
strategyqa_fewshot_file = "../StrategyQA/StrategyQA_examples/strategyqa_few_shot.json"

# Call the function to get everything we need in one go
data_assets = dataset_folders(strategyqa_base_folder, strategyqa_dataset_file, strategyqa_fewshot_file,  few_shot_type="static", tuning_subset_size=5)

# Unpack the dictionary into these variables for easy access
strategyqa_data_full = data_assets["data_full"]
strategyqa_data = data_assets["data_subset"] # This is the subset of 5 examples
shot_examples = data_assets["shot_examples"]
zero_shot_folder = data_assets["zero_shot_folder"]
few_shot_folder = data_assets["few_shot_folder"]
    

##### $Zero$ $Shot$ $Experiment$

In [ ]:
# Run the zero-shot experiment
print("\nStarting Zero-Shot Experiment")

zero_shot_results = run_experiment(
    model=model,
    tokenizer=tokenizer,
    data=strategyqa_data,
    shot_examples=shot_examples,
    model_config=model_config,  
    num_shots=0,
    batch_size=5
)

save_results_to_json(zero_shot_results, zero_shot_folder, model_file_name)

##### $Few$ $Shot$ $Experiment$ $-$ $Static$ $Shots$

In [ ]:
# Run the few-shot experiment with 3 shots
print("\nStarting 3-Shot Experiment")
# Run a 3-shot experiment
three_shot_results = run_experiment(
    model=model,
    tokenizer=tokenizer,
    data=strategyqa_data,
    shot_examples=shot_examples,
    model_config=model_config, # Use the same config
    num_shots=3,
    few_shot_type="static",  # "static" | "random" | "dynamic"
    retriever=None,          # Required if few_shot_type="dynamic"
    seed=42,
    batch_size=5
)

# Save the results with a different filename
save_results_to_json(three_shot_results, few_shot_folder, f"3shot_{model_file_name}")

##### $Few$ $Shot$ $Experiment$ $-$ $Random$ $Shots$

In [ ]:
# Create the folder paths for results and the folders if they do not exist
strategyqa_base_folder = '../StrategyQA/llm_predictions/random'
strategyqa_dataset_file = "../StrategyQA/StrategyQA_examples/strategyqa_evaluation.json" # Define the path to the strategyqa dataset
strategyqa_fewshot_file = "../StrategyQA/StrategyQA_examples/strategyqa_few_shot.json"

# Call the function to get everything we need in one go
data_assets = dataset_folders(strategyqa_base_folder, strategyqa_dataset_file, strategyqa_fewshot_file,  few_shot_type="static", tuning_subset_size=5)

# Unpack the dictionary into these variables for easy access
strategyqa_data_full = data_assets["data_full"]
strategyqa_data = data_assets["data_subset"] # This is the subset of 5 examples
shot_examples = data_assets["shot_examples"]
zero_shot_folder = data_assets["zero_shot_folder"]
few_shot_folder = data_assets["few_shot_folder"]
    

In [ ]:
# Run the few-shot experiment with 3 shots
print("\nStarting Random-3-Shot Experiment")
# Run a 3-shot experiment
three_shot_results = run_experiment(
    model=model,
    tokenizer=tokenizer,
    data=strategyqa_data,
    shot_examples=shot_examples,
    model_config=model_config, # Use the same config
    num_shots=3,
    few_shot_type="random",  # "static" | "random" | "dynamic"
    retriever=None,          # Required if few_shot_type="dynamic"
    seed=42,
    batch_size=5
)

# Save the results with a different filename
save_results_to_json(three_shot_results, few_shot_folder, f"3shot_{model_file_name}")

##### $Few$ $Shot$ $Experiment$ $-$ $Dynamic$ $Shots$

In [ ]:
# Create the folder paths for results and the folders if they do not exist
strategyqa_base_folder = '../StrategyQA/llm_predictions/dynamic'
strategyqa_dataset_file = "../StrategyQA/StrategyQA_examples/strategyqa_evaluation.json" # Define the path to the strategyqa dataset
strategyqa_fewshot_file = "../StrategyQA/StrategyQA_examples/strategyqa_few_shot.json"

# Call the function to get everything we need in one go
data_assets = dataset_folders(strategyqa_base_folder, strategyqa_dataset_file, strategyqa_fewshot_file,  few_shot_type="static", tuning_subset_size=5)

# Unpack the dictionary into these variables for easy access
strategyqa_data_full = data_assets["data_full"]
strategyqa_data = data_assets["data_subset"] # This is the subset of 5 examples
shot_examples = data_assets["shot_examples"]
zero_shot_folder = data_assets["zero_shot_folder"]
few_shot_folder = data_assets["few_shot_folder"]
    

In [ ]:
# Run the few-shot experiment with 3 shots
print("\nStarting Dynamic-3-Shot Experiment")

retriever_instance = DynamicRetriever(shot_examples)

# Run a 3-shot experiment
three_shot_results = run_experiment(
    model=model,
    tokenizer=tokenizer,
    data=strategyqa_data,
    shot_examples=shot_examples,
    model_config=model_config, # Use the same config
    num_shots=3,
    few_shot_type="dynamic",  # "static" | "random" | "dynamic"
    retriever=retriever_instance,          # Required if few_shot_type="dynamic"
    seed=42,
    batch_size=5
)

# Save the results with a different filename
save_results_to_json(three_shot_results, few_shot_folder, f"3shot_{model_file_name}")

In [ ]:
from huggingface_hub import model_info

# model_name = "mistralai/Mistral-7B-Instruct-v0.3"
info = model_info(model_name)

# Print description
print("Description:", info.cardData.get("description", "No description available"))

# Print any parameters listed (some models include recommended temperature/top_p)
parameters = info.cardData.get("parameters", {})
print("Parameters:", parameters)
